In [1]:
# Setup Imports
import os
import sys
import duckdb
import pandas as pd
from huggingface_hub import hf_hub_download

In [2]:
# Add project source code to path for grabbing functions
sys.path.append("../src")

In [3]:
from security_rules import (
    failed_logins,
    failed_count,
    failed_logins_by_ip,
    brute_force_candidates,
    privilege_escalations,
    account_lockouts,
    log_cleared,
    suspicious_linux_process,
    suspicious_windows_process,
    rdp_logins,
    ssh_root_logins,
)

Failed login attempts:
         AGENT_NAME                                            Content  \
0      UBUNTU_AGENT  sshd[2756135]: Failed password for invalid use...   
1      UBUNTU_AGENT  sshd[2653867]: Failed password for root from 2...   
2      UBUNTU_AGENT  sshd[2613573]: Failed password for root from 3...   
3      UBUNTU_AGENT  sshd[2617018]: Failed password for root from 3...   
4      UBUNTU_AGENT  sshd[895979]: Failed password for root from 21...   
...             ...                                                ...   
16655  UBUNTU_AGENT  sshd[2642903]: Failed password for invalid use...   
16656  UBUNTU_AGENT  sshd[2606192]: Failed password for invalid use...   
16657  UBUNTU_AGENT  sshd[867388]: Failed password for root from 21...   
16658  UBUNTU_AGENT  sshd[2667876]: Failed password for root from 2...   
16659  UBUNTU_AGENT  sshd[2582741]: Failed password for invalid use...   

                src_ip         dst_ip            entity_id         City  \
0      185.21

In [4]:
#connect to the database
con = duckdb.connect("../data/logs.duckdb")
#sample check
row_count = con.execute("SELECT COUNT(*) FROM logs").fetchone()[0]
print("Connected, total rows:", row_count)

Connected, total rows: 1920624


In [5]:
sample_df = con.execute("SELECT * FROM logs").fetchdf()
sample_df

,AGENT_NAME,Content,src_ip,dst_ip,entity_id,City,Country,date,day,hour,minute,second,timestamp
0,WINDOWS_AGENT,Microsoft-Windows-Security-Auditing\tEvent ID:...,172.20.82.33,172.20.82.33,Lemonade@172.20.82.33,Unknown,Unknown,2025-04-24 22:14:26,3,22,14,26,2025-04-24 22:14:26
1,WINDOWS_AGENT,Microsoft-Windows-Security-Auditing\tEvent ID:...,172.20.82.33,172.20.82.33,Lemonade@172.20.82.33,Unknown,Unknown,2025-04-25 06:53:30,4,6,53,30,2025-04-25 06:53:30
2,WINDOWS_AGENT,Microsoft-Windows-Security-Auditing\tEvent ID:...,172.20.82.33,172.20.82.33,Lemonade@172.20.82.33,Unknown,Unknown,2025-04-25 03:02:00,4,3,2,0,2025-04-25 03:02:00
3,UBUNTU_AGENT,Nisir_Ml_Agent[1045]: ...,172.20.0.42,172.20.0.42,ubuntu@172.20.0.42,Unknown,Unknown,2024-12-18 07:32:20,2,7,32,20,2024-12-18 07:32:20
4,UBUNTU_AGENT,logstash[323986]: Sending Logstash logs to /va...,172.20.0.39,172.20.0.39,ubuntu@172.20.0.39,Unknown,Unknown,2025-05-15 07:03:19,3,7,3,19,2025-05-15 07:03:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920619,UBUNTU_AGENT,systemd-resolved[621]: Using degraded feature ...,172.20.0.42,172.20.0.42,ubuntu@172.20.0.42,Unknown,Unknown,2025-05-13 19:29:51,1,19,29,51,2025-05-13 19:29:51
1920620,WINDOWS_AGENT,PowerShell\tEvent ID: 400\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-05-15 12:27:23,3,12,27,23,2025-05-15 12:27:23
1920621,WINDOWS_AGENT,Microsoft-Windows-Security-Auditing\tEvent ID:...,172.20.82.33,172.20.82.33,Lemonade@172.20.82.33,Unknown,Unknown,2025-05-14 01:58:15,2,1,58,15,2025-05-14 01:58:15
1920622,WINDOWS_AGENT,Microsoft-Windows-Security-Auditing\tEvent ID:...,172.20.82.33,172.20.82.33,Lemonade@172.20.82.33,Unknown,Unknown,2025-05-14 07:43:00,2,7,43,0,2025-05-14 07:43:00


In [ ]:
#  Failed Logins
failed_df = failed_logins(con)
print(f"Total failed logins: {len(failed_df)}")
display(failed_df.head(10))

# Failed Logins by IP
failed_by_ip_df = failed_logins_by_ip(con)
print(f"Top failed login source IPs ({len(failed_by_ip_df)} total):")
display(failed_by_ip_df.head(10))

# Total Failed Login Count
failed_count_df = failed_count(con)
print(f"Total failed login attempts: {int(failed_count_df.iloc[0, 0])}")

# Brute-Force Detection 
brute_force_df = brute_force_candidates(con)
print(f"Detected brute-force candidates: {len(brute_force_df)}")
display(brute_force_df.head(10))

# Privilege Escalation Events
priv_df = privilege_escalations(con)
print(f"Detected privilege escalations: {len(priv_df)}")
display(priv_df.head(10))

# Account Lockouts 
lockout_df = account_lockouts(con)
print(f"Account lockout events detected: {len(lockout_df)}")
display(lockout_df.head(10))

# Log Clearing / Tampering 
log_clear_df = log_cleared(con)
print(f"Log clearing/tampering events detected: {len(log_clear_df)}")
display(log_clear_df.head(10))

# Suspicious Processes (Linux)
susp_linux_df = suspicious_linux_process(con)
print(f"Suspicious Linux processes detected: {len(susp_linux_df)}")
display(susp_linux_df.head(10))

# Suspicious Processes (Windows)
susp_win_df = suspicious_windows_process(con)
print(f"Suspicious Windows processes detected: {len(susp_win_df)}")
display(susp_win_df.head(10))

# Remote Access / Root Logins
rdp_df = rdp_logins(con)
ssh_root_df = ssh_root_logins(con)
print(f"RDP login events: {len(rdp_df)} | Linux SSH root logins: {len(ssh_root_df)}")
display(rdp_df.head(10))
display(ssh_root_df.head(10))

print("\n✅ All detection rules executed successfully.")

#failed_df
#failed_by_ip_df
#failed_count_df
#brute_force_df
#priv_df
#lockout_df
#log_clear_df
#susp_linux_df
#susp_win_df
#rdp_df

Total failed logins: 16660


,AGENT_NAME,Content,src_ip,dst_ip,entity_id,City,Country,date,day,hour,minute,second,timestamp
0,UBUNTU_AGENT,sshd[2756135]: Failed password for invalid use...,185.213.165.107,172.21.35.175,Nisir@172.21.35.175,Tehran,Iran,2025-05-15 09:25:56,3,9,25,56,2025-05-15 09:25:56
1,UBUNTU_AGENT,sshd[2653867]: Failed password for root from 2...,218.92.0.177,172.21.35.175,Nisir@172.21.35.175,Unknown,China,2025-05-14 10:14:47,2,10,14,47,2025-05-14 10:14:47
2,UBUNTU_AGENT,sshd[2613573]: Failed password for root from 3...,36.110.15.2,172.21.35.175,Nisir@172.21.35.175,Unknown,China,2025-05-13 23:56:17,1,23,56,17,2025-05-13 23:56:17
3,UBUNTU_AGENT,sshd[2617018]: Failed password for root from 3...,36.110.15.2,172.21.35.175,Nisir@172.21.35.175,Unknown,China,2025-05-14 00:39:02,2,0,39,2,2025-05-14 00:39:02
4,UBUNTU_AGENT,sshd[895979]: Failed password for root from 21...,218.92.0.178,172.21.35.175,Nisir@172.21.35.175,Unknown,China,2025-04-24 18:56:27,3,18,56,27,2025-04-24 18:56:27
5,UBUNTU_AGENT,sshd[2758183]: Failed password for root from 1...,159.89.14.55,172.21.35.175,Nisir@172.21.35.175,Frankfurt am Main,Germany,2025-05-15 09:52:40,3,9,52,40,2025-05-15 09:52:40
6,UBUNTU_AGENT,sshd[872739]: Failed password for invalid user...,88.214.48.17,172.21.35.175,Nisir@172.21.35.175,New York,United States,2025-04-24 12:35:45,3,12,35,45,2025-04-24 12:35:45
7,UBUNTU_AGENT,sshd[862477]: Failed password for invalid user...,62.171.149.193,172.21.35.175,Nisir@172.21.35.175,Frankfurt am Main,Germany,2025-04-24 10:00:22,3,10,0,22,2025-04-24 10:00:22
8,UBUNTU_AGENT,sshd[2608208]: Failed password for root from 3...,36.110.15.2,172.21.35.175,Nisir@172.21.35.175,Unknown,China,2025-05-13 22:59:16,1,22,59,16,2025-05-13 22:59:16
9,UBUNTU_AGENT,sshd[2572642]: Failed password for invalid use...,92.118.39.61,172.21.35.175,Nisir@172.21.35.175,Dallas,United States,2025-05-13 13:47:19,1,13,47,19,2025-05-13 13:47:19


Top failed login source IPs (9 total):


,src_ip,attempts
0,172.20.82.21,55
1,172.20.0.39,33
2,172.20.82.33,6
3,159.89.14.55,4
4,172.20.82.28,3
5,218.92.0.178,3
6,172.21.35.175,3
7,5.9.0.0,1
8,172.20.82.26,1


Total failed login attempts: 109
Detected brute-force candidates: 12


...

,AGENT_NAME,Content,src_ip,dst_ip,entity_id,City,Country,date,day,hour,minute,second,timestamp
0,WINDOWS_AGENT,PowerShell\tEvent ID: 600\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-04-25 09:48:44,4,9,48,44,2025-04-25 09:48:44
1,WINDOWS_AGENT,PowerShell\tEvent ID: 403\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-05-14 12:47:45,2,12,47,45,2025-05-14 12:47:45
2,WINDOWS_AGENT,PowerShell\tEvent ID: 600\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-05-14 09:14:13,2,9,14,13,2025-05-14 09:14:13
3,WINDOWS_AGENT,PowerShell\tEvent ID: 403\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-05-14 06:49:44,2,6,49,44,2025-05-14 06:49:44
4,WINDOWS_AGENT,PowerShell\tEvent ID: 600\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-04-25 01:28:21,4,1,28,21,2025-04-25 01:28:21
5,WINDOWS_AGENT,PowerShell\tEvent ID: 600\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-05-14 06:49:43,2,6,49,43,2025-05-14 06:49:43
6,WINDOWS_AGENT,PowerShell\tEvent ID: 403\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-04-25 01:28:21,4,1,28,21,2025-04-25 01:28:21
7,WINDOWS_AGENT,PowerShell\tEvent ID: 600\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-04-25 01:28:20,4,1,28,20,2025-04-25 01:28:20
8,WINDOWS_AGENT,PowerShell\tEvent ID: 600\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-04-25 01:28:21,4,1,28,21,2025-04-25 01:28:21
9,WINDOWS_AGENT,PowerShell\tEvent ID: 600\tType: 4\tDescriptio...,172.20.82.21,172.20.82.21,DESKTOP-Q0KDN51@172.20.82.21,Unknown,Unknown,2025-05-14 12:47:45,2,12,47,45,2025-05-14 12:47:45


RDP login events: 0 | Linux SSH root logins: 30819


,AGENT_NAME,Content,src_ip,dst_ip,entity_id,City,Country,date,day,hour,minute,second,timestamp


,AGENT_NAME,Content,src_ip,dst_ip,entity_id,City,Country,date,day,hour,minute,second,timestamp
0,UBUNTU_AGENT,sshd[2567023]: pam_unix(sshd:auth): authentica...,172.21.35.175,172.21.35.175,Nisir@172.21.35.175,Unknown,Unknown,2025-05-13 12:24:21,1,12,24,21,2025-05-13 12:24:21
1,UBUNTU_AGENT,sshd[2757072]: Connection closed by authentica...,159.89.14.55,172.21.35.175,Nisir@172.21.35.175,Frankfurt am Main,Germany,2025-05-15 09:38:29,3,9,38,29,2025-05-15 09:38:29
2,UBUNTU_AGENT,sshd[2602960]: Connection closed by authentica...,36.110.15.2,172.21.35.175,Nisir@172.21.35.175,Unknown,China,2025-05-13 21:51:49,1,21,51,49,2025-05-13 21:51:49
3,UBUNTU_AGENT,sshd[910786]: PAM 2 more authentication failur...,172.21.35.175,172.21.35.175,Nisir@172.21.35.175,Unknown,Unknown,2025-04-24 23:11:55,3,23,11,55,2025-04-24 23:11:55
4,UBUNTU_AGENT,sshd[2653867]: Failed password for root from 2...,218.92.0.177,172.21.35.175,Nisir@172.21.35.175,Unknown,China,2025-05-14 10:14:47,2,10,14,47,2025-05-14 10:14:47
5,UBUNTU_AGENT,sshd[2753290]: PAM 2 more authentication failu...,172.21.35.175,172.21.35.175,Nisir@172.21.35.175,Unknown,Unknown,2025-05-15 09:02:11,3,9,2,11,2025-05-15 09:02:11
6,UBUNTU_AGENT,sshd[2613573]: Failed password for root from 3...,36.110.15.2,172.21.35.175,Nisir@172.21.35.175,Unknown,China,2025-05-13 23:56:17,1,23,56,17,2025-05-13 23:56:17
7,UBUNTU_AGENT,sshd[860422]: pam_unix(sshd:auth): authenticat...,172.21.35.175,172.21.35.175,Nisir@172.21.35.175,Unknown,Unknown,2025-04-24 09:32:10,3,9,32,10,2025-04-24 09:32:10
8,UBUNTU_AGENT,sshd[2617018]: Failed password for root from 3...,36.110.15.2,172.21.35.175,Nisir@172.21.35.175,Unknown,China,2025-05-14 00:39:02,2,0,39,2,2025-05-14 00:39:02
9,UBUNTU_AGENT,sshd[2760504]: pam_unix(sshd:auth): authentica...,172.21.35.175,172.21.35.175,Nisir@172.21.35.175,Unknown,Unknown,2025-05-15 10:22:17,3,10,22,17,2025-05-15 10:22:17



✅ All detection rules executed successfully.


In [7]:
# storing the model path into a variable
model_path = hf_hub_download(
    repo_id="TheBloke/Llama-2-13B-chat-GGUF",
    filename="llama-2-13b-chat.Q5_K_M.gguf"
)

In [8]:
# Load the LLM
from llama_cpp import Llama, llama_print_system_info

llm = Llama(
    model_path= model_path,
    n_ctx=4096,
    n_threads=12,
    n_gpu_layers=42,
    n_batch=512
)

#llama_print_system_info()
print("LLaMA 13B model loaded!")

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4070, compute capability 8.9, VMM: yes
llama_model_load_from_file_impl: using device CUDA0 (NVIDIA GeForce RTX 4070) - 11094 MiB free
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from C:\Users\Ishaq\.cache\huggingface\hub\models--TheBloke--Llama-2-13B-chat-GGUF\snapshots\4458acc949de0a9914c3eab623904d4fe999050a\llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.emb

LLaMA 13B model loaded!


CUDA : ARCHS = 890 | USE_GRAPHS = 1 | PEER_MAX_BATCH_SIZE = 128 | CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | LLAMAFILE = 1 | OPENMP = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'general.name': 'LLaMA v2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '5120', 'llama.block_count': '40', 'llama.feed_forward_length': '13824', 'llama.attention.head_count': '40', 'tokenizer.ggml.eos_token_id': '2', 'general.file_type': '17', 'llama.attention.head_count_kv': '40', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0'}
Using fallback chat format: llama-2


In [ ]:
# Testing to get insight into a subset of the logs.
summary = failed_df.head(5).to_string(index=False)
prompt = f"""
You are a SOC (Security Operations Center) analyst.
Here are some recent failed login events:

{summary}

Explain what potential threat this indicates and what steps should I take as an analyst.
"""
response = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": "You are an expert SOC analyst."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=400,
    temperature=0.6,
)

print("🧠 LLaMA Analysis:\n")
print(response["choices"][0]["message"]["content"])

llama_perf_context_print:        load time =    6142.07 ms
llama_perf_context_print: prompt eval time =    6140.57 ms /   844 tokens (    7.28 ms per token,   137.45 tokens per second)
llama_perf_context_print:        eval time =   35159.58 ms /   399 runs   (   88.12 ms per token,    11.35 tokens per second)
llama_perf_context_print:       total time =   41563.47 ms /  1243 tokens


🧠 LLaMA Analysis:

  As an expert SOC analyst, I can analyze the failed login events you provided and identify potential threats and recommended steps to take.

Based on the event logs, there are several failed login attempts from different IP addresses, all attempting to access the same Ubuntu server with the username "root" and the password "nisir". The failed login attempts were made from various geolocations, including Iran, China, and the United States.

Here are the potential threats that could be indicated by these failed login events:

1. Brute-force attack: It is possible that an attacker is using a brute-force attack to try to guess the password for the "root" username. The fact that multiple IP addresses are attempting to log in with the same password suggests that this could be a coordinated attack.
2. Password spraying attack: Another possibility is that the attacker is using a password spraying attack, where they are trying multiple passwords in hopes of guessing the corr

In [14]:
# Failed logins: {len(failed_df)}
#failed_by_ip_df
#failed_count_df
#Brute-force attempts: {len(brute_force_df)}
#Privilege escalations: {len(priv_df)}
#lockout_df
#log_clear_df
#Suspicious processes (Linux): {len(susp_linux_df)}
#Suspicious processes (Windows): {len(susp_win_df)}
#rdp_df
combined_summary_1= f"""
Failed logins: {len(failed_df)}
Brute-force attempts: {len(brute_force_df)}
Privilege escalations: {len(priv_df)}
Suspicious processes (Linux): {len(susp_linux_df)}
Suspicious processes (Windows): {len(susp_win_df)}
"""

prompt = f"""
You are an AI SOC analyst. Based on the following summary, explain what overall security threat
this system might be experiencing and what should be prioritized.

{combined_summary_1}
"""

response = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": "You are an expert SOC analyst specialized in threat correlation."},
        {"role": "user", "content": prompt},
    ],
    max_tokens=600,
    temperature=0.5,
)

print("Combined Threat Analysis:")
print(response["choices"][0]["message"]["content"])

Llama.generate: 12 prefix-match hit, remaining 123 prompt tokens to eval
llama_perf_context_print:        load time =    6142.07 ms
llama_perf_context_print: prompt eval time =     624.03 ms /   123 tokens (    5.07 ms per token,   197.10 tokens per second)
llama_perf_context_print:        eval time =   27539.75 ms /   487 runs   (   56.55 ms per token,    17.68 tokens per second)
llama_perf_context_print:       total time =   28481.39 ms /   610 tokens


Combined Threat Analysis:
  As an AI SOC analyst, I have analyzed the provided summary and identified the following overall security threat to the system:

1. Brute-force attacks: The 12 brute-force attempts detected suggest a potential attempt to guess or crack passwords or encryption keys. This is a high-priority threat as it could lead to unauthorized access or data breaches.
2. Privilege escalation: The 58993 privilege escalation attempts detected indicate a potential attempt to gain elevated access or control over the system. This is a high-priority threat as it could lead to a full system compromise.
3. Suspicious processes: The 24 suspicious Linux processes and 32 suspicious Windows processes detected suggest a potential infection or malicious activity. This is a medium-priority threat as it could lead to data breaches or system compromise.

Based on the above analysis, the following should be prioritized:

1. Brute-force attack prevention: Implementing multi-factor authenticati

In [15]:
def ask_ai_about_logs(df, question):
    summary = df.head(10).to_string(index=False)
    prompt = f"""
    You are a cybersecurity analyst AI. Analyze the following log data and answer this question:

    Log sample:
    {summary}

    Question: {question}
    """
    response = llm.create_chat_completion(
        messages=[
            {"role": "system", "content": "You are an experienced SOC analyst."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=400,
        temperature=0.6,
    )
    return response["choices"][0]["message"]["content"]

# Example query
question = "Based on the Windows event logs, do we see evidence of repeated intrusion attempts originating from a single IP address, or do the patterns suggest multiple distinct attackers?"
print(ask_ai_about_logs(failed_df, question))

Llama.generate: 12 prefix-match hit, remaining 1602 prompt tokens to eval
llama_perf_context_print:        load time =    6142.07 ms
llama_perf_context_print: prompt eval time =   10447.42 ms /  1602 tokens (    6.52 ms per token,   153.34 tokens per second)
llama_perf_context_print:        eval time =   35257.75 ms /   399 runs   (   88.37 ms per token,    11.32 tokens per second)
llama_perf_context_print:       total time =   45937.74 ms /  2001 tokens


  Based on the log data provided, it appears that there are multiple distinct attackers attempting to gain access to the system. Here's why:

1. Different IP addresses: Each failed login attempt has a different IP address associated with it (185.213.165.107, 218.92.0.177, 36.110.15.2, 36.110.15.2 again, 218.92.0.178, 159.89.14.55, 88.214.48.17, 62.171.149.193, and 92.118.39.61). This suggests that the attacks are coming from multiple sources.
2. Different user names: The attackers are using different user names each time they attempt to log in (e.g., "Nisir@172.21.35.175", "root", "super", "sc", and "polkadot"). This further supports the idea that the attacks are being carried out by multiple distinct attackers.
3. Different times and days: The failed login attempts occur on different days and at different times (e.g., May 13th, May 14th, May 15th, April 24th). This suggests that the attacks are not being carried out by a single attacker working at a consistent pace.
4. Different count